# Research Track 2 - Assignment 2
## Implementation of a user interface with ROS in a Jupyter notebook for Assignment 2

## Imorting necessary libraries and variables

In [ ]:
# ROS libraries
import rospy
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
import actionlib
import actionlib.msg
from rt1_a2_2023.msg import PlanningAction, PlanningGoal
from rt1_a2_2023.msg import TargetPosition, Abort, RobotPositionVelocity
from std_srvs.srv import *
from rt1_a2_2023.srv import LastTarget, RobotToTarget

# ipywidgets libraries
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox 
from IPython.display import display 
from IPython.display import clear_output

# matplotlib libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tf.transformations import quaternion_matrix

################################

# global variables
global temp_status
global have_goal
global goal_reached

# Initialize the publisher
global publisher
publisher = rospy.Publisher("/RobotPositionVelocity", RobotPositionVelocity, queue_size=1)

global action_client

global now_x
global now_y
x_array = []
y_array = []

canceled_goal_x = []
canceled_goal_y = []

new_given_goal_x = []
new_given_goal_y = []

number_reached_goals = 0

################################


## interface logic

In [ ]:

# def interface_behaviour(selector):
#     if selector:
#         x_coord.disabled=False
#         y_coord.disabled=False
#         submit_b.disabled=False
#         cancel_b.disabled=True

#     else:
#         x_coord.disabled=True
#         y_coord.disabled=True
#         submit_b.disabled=True
#         cancel_b.disabled=False

## callback function for the subscriber

In [ ]:
def publish_message(msg):
    global publisher
    # get the current position of the robot from the msg in the topic /odom.
    position = msg.pose.pose.position
    linear_velocity = msg.twist.twist.linear

    now_x = position.x
    now_y = position.y

    x_arrey.append(now_x)
    y_arrey.append(now_y)

    #define the message that will be published
    pos_vel = RobotPositionVelocity()
    pos_vel.x = position.x
    pos_vel.y = position.y
    pos_vel.v_x = linear_velocity.x
    pos_vel.v_y = linear_velocity.y

    # publish the message
    publisher.publish(pos_vel)

## Cancel button callback function

In [ ]:
def cancel_button_clicked_callback(b):
    with output:
        if have_goal:
            canceled_goal_x.append(rospy.get_param('/des_pos_x'))
            canceled_goal_y.append(rospy.get_param('/des_pos_y'))
            # cancel the goal
            have_goal = 0
            action_client.cancel_goal()
            rospy.loginfo("Goal canceled")
            print("Target cancelled")
        elif not have_goal:
            rospy.loginfo("No goal to cancel")

## Change button callback function

In [ ]:
def change_button_clicked_callback(b):
    with output:
        if have_goal:
            # cancel the goal first
            have_goal = 0
            action_client.cancel_goal()
            rospy.loginfo("Goal canceled")
            # get the new goal from the client
            rospy.set_param('/des_pos_x', c1_slider.value)
            rospy.set_param('/des_pos_y', c2_slider.value)
            # change the goal and send it to the action server
            new_target_x = c1_slider.value
            new_target_y = c2_slider.value
            rospy.loginfo("New goal is: x = %f, y = %f", new_target_x, new_target_y)
            goal_new = rt1_a2_2023.msg.PlanningGoal()
            goal_new.target_pose.pose.position.x = new_target_x
            goal_new.target_pose.pose.position.y = new_target_y
            new_given_goal_x.append(new_target_x)
            new_given_goal_y.append(new_target_y)

            # send new goal to the action server
            action_client.send_goal(goal_new)
            have_goal = 1

            print("New target set: X =",  goal_new.target_pose.pose.position.x, "Y =", goal_new.target_pose.pose.position.y)

## Info button callback function

In [ ]:
def info_button_clicked_callback(b):
    with output:
        print("Current position: X =", now_x, "Y =", now_y)
        given_goals_list = list(zip(new_given_goal_x, new_given_goal_y))
        print("Given goals: ", given_goals_list)
        canceled_goals_list = list(zip(canceled_goal_x, canceled_goal_y))
        print("Canceled goals: ", canceled_goals_list)

## Slider for define goals 

In [ ]:
c1_slider = widgets.IntSlider(
    value = 1,
    min = -10,
    max = 10,
    step = 1,
    description = 'X_goal:',
    disabled = False
    continuous_update = False,
    orientation = 'horizontal',
    readout = True,
    readout_format = 'd'
)

c2_slider = widgets.IntSlider(
    value = 0,
    min = -10,
    max = 10,
    step = 1,
    description = 'Y_goal:',
    disabled = False,
    continuous_update = False,
    orientation = 'vertical',
    readout = True,
    readout_format = 'd'
)

instructions_html = """
<h3 style="color: blue;">Choose the goal:</h3>
"""

instructions_widget1 = widgets.HTML(value = instructions_html)
display(instructions_widget1, c1_slider, c2_slider)

## widget description

In [ ]:
instructions_html2 = """
<h3 style="color: blue;">Instructions:</h3>
<p style = "font-size: 16px;">1. Choose what you want to do next:</p>
<p style = "font-size: 16px;">2. Choose Cancel to Cancel the target.</p>
<p style = "font-size: 16px;">3. Choose Change to change the target.(determine the target in the slider above the page before choosing Change)</p>
<p style="color: red; font-family: 'Times New Roman', serif; font-size: 18px;"></p>
"""

instructions_widget2 = widgets.HTML(value=instructions_html2)

################################

## Choice Maker
change_button = Button(description='Change', 
                       layout=Layout(width='auto', align="center", grid_area='b1'), 
                       style=ButtonStyle(button_color='green'))

cancel_button = Button(description='Cancel', 
                       layout=Layout(width='auto', grid_area='b2'), 
                       style=ButtonStyle(button_color='red'))

info_button = Button(description='General Info', 
                     layout=Layout(width='auto', align="center", grid_area='b1'), 
                     style=ButtonStyle(button_color='lightblue'))

output = widgets.Output()

change_button.on_click(change_button_clicked_callback)
cancel_button.on_click(cancel_button_clicked_callback)
info_button.on_click(info_button_clicked_callback)

buttons_box = VBox([HBox([info_button]),HBox([change_button,cancel_button])])
output_box = VBox([buttons_box, output])

display(instructions_widget2 , output_box)

## Initialize the node and subscribe to the topic /odom channel to get the current position of the robot

In [ ]:
# Initialize the node
rospy.init_node('DefineActionClient')

# log the initialization of the node
rospy.loginfo("Node initialized successfully")

# Initialize the subscriber
global subscriber
subscriber = rospy.Subscriber("/odom", Odometry, publish_message)

# Initialize the action client
action_client = actionlib.SimpleActionClient('/reaching_goal', rt1_a2_2023.msg.PlanningAction)
action_client.wait_for_server()
rospy.loginfo("Action client initialized successfully")

now_x = rospy.get_param('/des_pos_x')
now_y = rospy.get_param('/des_pos_y')

rospy.loginfo("Current position: x = %f, y = %f", now_x, now_y)

# Visualization

## Path Visualizer

In [ ]:
class Visualiser:

    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = self.ax.plot([], [], 'ro', label='Robot Position')  # Target's Position Plot
        self.goal_ln, = self.ax.plot([], [], 'bo', markersize=10, label='Robot Goal Position')  # Robot's Position Data Arrays
        self.cancel_goal_ln, = self.ax.plot([], [], 'rx', markersize=10, label='Robot Cancelled Goal Position')  # Robot's Position Cancel goals Data Arrays
        self.x_array, self.y_array = [], []
        self.Goal_x, self.Goal_y = [], []

    def plot_init(self):
        self.ax.set_title("Robot Position", fontsize=15, fontweight='bold')  # Set Plot Axis Labels
        self.ax.set_xlabel("X [m]", fontsize=10, fontweight="bold")
        self.ax.set_ylabel("Y [m]", fontsize=10, fontweight="bold")
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        self.ax.grid(True)
        self.ax.legend()
        return self.ln, self.goal_ln
    
    def odom_callback(self, msg):
        self.x_array.append(msg.pose.pose.position.x)
        self.y_array.append(msg.pose.pose.position.y)
        x_t = rospy.get_param('/des_pos_x')
        y_t = rospy.get_param('/des_pos_y')
        self.Goal_x.append(x_t)
        self.Goal_y.append(y_t)
        self.cancelled_x = np.array(canceled_goal_x)
        self.cancelled_y = np.array(canceled_goal_y)

    def update_plot(self, frame):
        self.ln.set_data(self.x_array, self.y_array)
        self.goal_ln.set_data(self.Goal_x, self.Goal_y)
        self.cancel_goal_ln.set_data(self.cancelled_x, self.cancelled_y)
        return self.ln, self.goal_ln, self.cancel_goal_ln